In [111]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
import os
import shutil
import subprocess
import glob

#Define the attributes of a grain: x,y,r,tesselation area and stress component s22:
class grain:
    def __init__(self, x_, y_, r_, dt_area,s22):
        self.x = x_
        self.y = y_
        self.r = r_
        self.dt_area = dt_area
        self.s22 = s22

#Define the class and use the scipy.spatial.Voronoi to calculate the voronoi area 
#the input is the dataframe read from DEM*.dat
#the output is first the list of grain attributes and second the packing density
class Voronoi:
    
    def __init__(self,data):
        self.data = data
    def grains(self):
        from scipy.spatial import Voronoi, voronoi_plot_2d
        radii = self.data[1]
        nbgrains = len(radii)
        x = self.data[2]
        y = self.data[3]
        M22 = self.data[26]
        points = np.vstack((x, y)).T
        vor = Voronoi(points)
        
        t_area = 0 # Tesselation area in total
        g_area = 0 # Grain area in total
        # creating list        
        grains = [] 

        for i in range(len(vor.regions)):
            dt_area = 0 # Tesselation area per cell
            dg_area = 0 # Grain area per cell
            for j in range(len(vor.regions[i])):
                 # Find the index of the point inside of the current voro pt_index:
                pt_index = int(np.argwhere(vor.point_region == i))
                if j == len(vor.regions[i])-1: k = 0
                else: k = j+1
                #When one of the vertice is out of the domain, disgard the cell and the grain inside;
                if vor.regions[i][j]==-1: 
                    dt_area = 0 
                    dg_area = 0 
                    break
                else:
                    next_indice = vor.regions[i][k]
                    current_indice = vor.regions[i][j]
                    next_x = vor.vertices[next_indice][0]
                    next_y = vor.vertices[next_indice][1]
                    current_x = vor.vertices[current_indice][0]
                    current_y = vor.vertices[current_indice][1]
                    dx = next_x - current_x
                    dy = next_y - current_y
                    edge = np.sqrt(dx*dx+dy*dy)
                    #Discard stretchy cells whose edge is longer than 4 times the inside grain radius 
                    if edge > (4*radii[pt_index]):
                        dt_area=0 
                        dg_area = 0
                        break
                    else:
                        dt_area = dt_area + (current_x * next_y - current_y * next_x)
                        dg_area =  math.pi * radii[pt_index] * radii[pt_index]  
                        
            #The tesselation area for each grain whose index is pt_index
            dt_area = math.fabs(dt_area)/2
            #Append the current grain attributes to the list grains[]
            if dt_area !=0: 
                grains.append(grain(x[pt_index], y[pt_index],radii[pt_index],dt_area,M22[pt_index]/dt_area))
            else: 
                grains.append(grain(x[pt_index], y[pt_index],radii[pt_index],0.,0.))
            g_area = g_area + dg_area
            t_area = t_area + dt_area

        #calculate the packing density
        void_area = t_area - g_area
        density = g_area / t_area
        return [grains,density]

In [128]:
## Normalized stress component s11 by the maximum initial s11 at base of the flowfront
#the aspect ratio and volume studied
condition = ["dcrR"]
volume= 2
a= "02"

#The initial time-step
ini_dat = "DEM000000.dat"


for c in condition:
#     while volume <6:
        v=str(volume)        
         #path to the outpufiles and excutable files(current location);
        output_path = "/home/amber/Documents/2d-lbm-dem-analysis/2d-lbm-dem/Analysis/a"+a+"/code/contactforces/scipy.spatial.Voronoi/"
        
        #path to the inputfiles(initial DEM*.dat);
        input_path_ini = "/home/amber/Documents/2d-lbm-dem-analysis/2d-lbm-dem/Analysis/a02/"+c+"_v"+v+"00dat/"

        #path to the inputfiles(DEM*.dat at time-step corresponding to 3tau_c for different volumes);
        input_path_3tc= "/home/amber/Documents/2d-lbm-dem-analysis/2d-lbm-dem/Analysis/a"+a+"/code/contactforces/"
 
                
## Process the initial DEM*.dat and obtain the maximum initial s22 at base of flowfront:max_s22;
        # Copy the corresponding initial DEM*.dat to output_path;
        filename = input_path_ini+ini_dat
        os.chdir(output_path)
        file_ini = shutil.copyfile(filename,"./"+c+"-v"+v+"_ini.dat")
        
         #calculate density and grain attributes using class Voronoi
        data_ini = pd.read_csv(file_ini,header= None, sep='\t')
        ini = Voronoi(data_ini)
        ini_density = ini.grains()[1]
        ini_grains=ini.grains()[0]
        print ("The initial packing density of a="+a+" V="+v+"0000cm3 ={}".format(ini_density))
        #determine the flowfront location
        max_radius = data_ini[1].max()
        max_x = data_ini[2].max()
        flowfront_x = max_x - 30*max_radius
        max_s22 = 0.
        #calculate max_s22
        for i in range(len(data_ini[1])):
            x_ = ini_grains[i].x
            y_ = ini_grains[i].y
            s22_ = ini_grains[i].s22
            if y_ <= max_radius :
                if s22_>max_s22:
                    max_s22=s22_
        print(max_s22)

## Process the 3tau_c DEM*.dat and obtain the normalized s22: nmlz_s22 at base of flowfront;
         # List of all the files at 3tauc_c;
        os.chdir(input_path_3tc)     
        files = filter(os.path.isfile, os.listdir('./')) 
        # Copy the corresponding DEM*.dat at 3 tau_c to output_path;
        for doc in files:
            if doc.startswith(c+"_v"+v):
                file_3tc = shutil.copyfile(doc,output_path+c+"-v"+v+"_3tc.dat")  
        os.chdir(output_path) 
        
        data_3tc = pd.read_csv(file_3tc,header= None, sep='\t')
        
        #determine the flowfront location
        max_radius = data_3tc[1].max()
        max_x = data_3tc[2].max()
        flowfront_x = max_x - 30*max_radius
        flowfront_l = 30*max_radius
        #calculate density and class grain using class Voronoi
        at_3tc = Voronoi(data_3tc)
        at_3tc_density = at_3tc.grains()[1]
        at_3tc_grains = at_3tc.grains()[0]
        print ("The packing density of a="+a+" V="+v+"0000cm3 at 3 tau_c={}".format(at_3tc_density))
        
        #Writing the stress compnent to txt
        output = c+"-v"+v+"base.txt"
        f1 = open(output, "w")
        for i in range(len(data_3tc[1])):
        # x_, y_, r_, dt_area,s22
         # x_, y_, r_, dt_area,s22
            x_ = at_3tc_grains[i].x
            y_ = at_3tc_grains[i].y
            s22_ = at_3tc_grains[i].s22
            if x_ >= flowfront_x and y_ <= max_radius:
                nmlz_x= (x_-flowfront_x)/flowfront_l
                if s22_!=0:         
                    nmlz_s22= s22_/max_s22
                    print(nmlz_x,nmlz_s22)
                    f1.write("%f %f \n"% (nmlz_x,nmlz_s22))   
                else:
                    f1.write("%f %f \n"% (nmlz_x,0))   
        f1.close()

        
        
      

The initial packing density of a=02 V=20000cm3 =0.8381093150363107
0.0
The packing density of a=02 V=20000cm3 at 3 tau_c=0.8318615995026
